In [ ]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf

In [ ]:
symbol = 'BAC'
bac = yf.download(symbol, start='2022-01-01', interval='1d')
bac

In [ ]:
import matplotlib.pyplot as plt
plt.plot(bac.index, bac['Close'])

In [ ]:
data = bac.copy()
data['SMA1'] = data['Close'].rolling(window=42).mean()
data['SMA2'] = data['Close'].rolling(window=252).mean()
data[['Close', 'SMA1', 'SMA2']].plot(figsize=(10, 6))
plt.show()


In [ ]:
import ta.momentum
import ta.trend

data['RSI'] = ta.momentum.RSIIndicator(data['Close']).rsi()
macd = ta.trend.MACD(data['Close'])
data['MACD'] = macd.macd()
data['MACD_signal'] = macd.macd_signal()

data.dropna(inplace=True)

data

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error

train_size = int(0.6 * len(data))
train = data.iloc[:train_size]
test_size = int(0.2 * len(data))
test = data.iloc[train_size:train_size+test_size]
validate = data.iloc[train_size+test_size:]

features = ['SMA1', 'SMA2', 'RSI', 'MACD', 'MACD_signal']
target = 'Close'

In [ ]:
def walk_forward(train: pd.DataFrame, test: pd.DataFrame, features, target, validate: pd.DataFrame):
    predictions = []
    history = train.copy()
    
    X_train = history[features].dropna()
    y_train = history[target].dropna()
    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = test[features].dropna()
    yhat = model.predict(X_test)
    predictions.extend(yhat)
    history = pd.concat([history, test.iloc[:len(yhat)]], ignore_index=True)

    future_predictions = []

    X_test = validate[features].dropna()
    yhat = model.predict(X_test)
    future_predictions.extend(yhat)

    new_row = validate.copy()
    new_row[target] = yhat[0]
    history = pd.concat([history, new_row], ignore_index=True)
        
    return predictions, future_predictions

predictions, future_predictions = walk_forward(train, test, features, target, validate)

In [ ]:
mse = mean_squared_error(test['Close'], predictions)
print(f'Mean Squared Error: {mse}')

plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual')
plt.plot(test.index, predictions, label='Predicted')
plt.legend()
plt.show()

In [ ]:
test['Predicted'] = predictions
validate['Predicted'] = future_predictions
plt.figure(figsize=(14, 7))

plt.plot(data.index, data['Close'], label='Actual')
plt.plot(test.index, test['Predicted'], label='Predicted')
plt.plot(validate.index, validate['Predicted'], label='Future Predicted')
plt.legend()
plt.show()